In [2]:
from api.db import stat_db, crawl_db, relation_db, CrawlCollections, StatCollections, RelationCollections, DESCENDING
import pandas as pd
import numpy as np
import re
from datetime import datetime

In [ ]:
link_cursor = crawl_db[CrawlCollections.REDDIT_LINK].find({'data.subreddit': 'NEO'})
link_df = pd.DataFrame([item['data'] for item in link_cursor], columns=['author', 'score'])
grouped_df = link_df.groupby('author', as_index=False)
aggregate_df = grouped_df['score'].agg([np.sum, np.mean, np.size]).reset_index().sort_values(by='sum', ascending=False)
list(aggregate_df[:10])
comment_cursor = crawl_db[CrawlCollections.REDDIT_COMMENT].find({'data.subreddit': 'NEO'})
comment_df = pd.DataFrame([item['data'] for item in comment_cursor], columns=['author', 'score'])
grouped_df = comment_df.groupby('author', as_index=False)
aggregate_df = grouped_df['score'].agg([np.sum, np.mean, np.size]).reset_index().sort_values(by='sum', ascending=False)
aggregate_df[aggregate_df['author'] != '[deleted]']

In [20]:
def get_df(col, sr_name, columns, query=None, drop_deleted=True):
    """从数据库中加载dataframe"""
    if query is None:
        query = {}
    query['data.subreddit'] = sr_name
    cs = crawl_db[col].find(query)
    df = pd.DataFrame([item['data'] for item in cs], columns=columns)
    if drop_deleted:
        df = df[df['author'] != '[deleted]']
    return df

In [ ]:
latest_record = crawl_db[CrawlCollections.REDDIT_LINK].find({}).sort([('data.created', DESCENDING)])
t = latest_record[1]['data']['created'] - 180 * 24 * 3600
ldf = get_df('NEO', ['id', 'created'], {'data.created': {'$gt': t}}, drop_deleted=False)
ldf.created = ldf['created'].apply(lambda x: datetime.fromtimestamp(x))
ldf.set_index(ldf.created, inplace=True)
ldf['num'] = 1
ldf.drop('created', axis=1)
df = ldf.resample('D').sum().fillna(0)
data = df.reset_index().to_json(orient='values')
stat_db['subreddit_time_distribute'].find_one_and_update({'_id': 'NEO', 'type': 'link'},{'$set': {'data': data}}, upsert=True)
df = get_df(CrawlCollections.REDDIT_LINK, 'NEO', ['id', 'title', 'created'], drop_deleted=False)
p = re.compile(r'\b(Zeepin | ZPT)\b', re.I | re.M )
df2 = df[df['title'].str.contains(p)]

# df.created = df['created'].apply(lambda x: datetime.fromtimestamp(x))
df.set_index(df.created, inplace=True)
df.drop('created', axis=1)
temp = df.resample('D')

df2 = temp.agg({'id': lambda x: list(x), 'title': lambda x: len(x)})
data = df2.reset_index().to_json(orient='values')

print(data)
df = get_df(CrawlCollections.REDDIT_COMMENT, 'NEO', ['id', 'body', 'created'], drop_deleted=False)
p = re.compile(r'\b(NEO)\b', re.I | re.M)
df2 = df[df['body'].str.contains(p)]
df2.index.size


# 分析key user的历史足迹

In [24]:
cs = crawl_db[CrawlCollections.REDDIT_COMMENT].find({'data.author': 'flygoing'})
df = pd.DataFrame([i['data'] for i in cs], columns=['author', 'subreddit', 'created'])

[{'data': '[[1517097600000,1]]', 'subreddit': 'AirSwap'},
 {'data': '[[1498780800000,3.0],[1498867200000,1.0],[1498953600000,4.0],[1499040000000,17.0],[1499126400000,21.0],[1499212800000,3.0],[1499299200000,3.0],[1499385600000,13.0],[1499472000000,6.0],[1499558400000,1.0],[1499644800000,0.0],[1499731200000,4.0],[1499817600000,5.0],[1499904000000,0.0],[1499990400000,4.0],[1500076800000,10.0],[1500163200000,4.0],[1500249600000,7.0],[1500336000000,2.0],[1500422400000,1.0],[1500508800000,9.0],[1500595200000,4.0],[1500681600000,5.0],[1500768000000,1.0],[1500854400000,2.0],[1500940800000,2.0],[1501027200000,9.0],[1501113600000,4.0],[1501200000000,3.0],[1501286400000,0.0],[1501372800000,0.0],[1501459200000,6.0],[1501545600000,11.0],[1501632000000,5.0],[1501718400000,4.0],[1501804800000,2.0],[1501891200000,1.0],[1501977600000,0.0],[1502064000000,0.0],[1502150400000,2.0],[1502236800000,8.0],[1502323200000,8.0],[1502409600000,4.0],[1502496000000,4.0],[1502582400000,5.0],[1502668800000,2.0],[1502

In [26]:

df['num'] = 1
grouped_df = df.groupby('subreddit', as_index=False).agg({'num': np.sum})
general_df = grouped_df.sort_values(by='num', ascending=False).to_json(orient='values')


[{'data': '[[1517097600000,1]]', 'subreddit': 'AirSwap'},
 {'data': '[[1498780800000,3.0],[1498867200000,1.0],[1498953600000,4.0],[1499040000000,17.0],[1499126400000,21.0],[1499212800000,3.0],[1499299200000,3.0],[1499385600000,13.0],[1499472000000,6.0],[1499558400000,1.0],[1499644800000,0.0],[1499731200000,4.0],[1499817600000,5.0],[1499904000000,0.0],[1499990400000,4.0],[1500076800000,10.0],[1500163200000,4.0],[1500249600000,7.0],[1500336000000,2.0],[1500422400000,1.0],[1500508800000,9.0],[1500595200000,4.0],[1500681600000,5.0],[1500768000000,1.0],[1500854400000,2.0],[1500940800000,2.0],[1501027200000,9.0],[1501113600000,4.0],[1501200000000,3.0],[1501286400000,0.0],[1501372800000,0.0],[1501459200000,6.0],[1501545600000,11.0],[1501632000000,5.0],[1501718400000,4.0],[1501804800000,2.0],[1501891200000,1.0],[1501977600000,0.0],[1502064000000,0.0],[1502150400000,2.0],[1502236800000,8.0],[1502323200000,8.0],[1502409600000,4.0],[1502496000000,4.0],[1502582400000,5.0],[1502668800000,2.0],[1502

In [32]:
arr_data = []
# df['created'] = df['created'].apply(lambda x: datetime.fromtimestamp(x))
df.set_index(df.created, inplace=True)
df.drop('created', axis=1)
d = df['subreddit'].astype('category')
for subreddit in d.cat.categories:
    temp_df = df[df['subreddit'] == subreddit]
    resampled_df = temp_df.resample('D').sum().fillna(0)
    json_data = resampled_df.reset_index().to_json(orient='values')
    arr_data.append({'subreddit': subreddit, 'data': json_data})

[{'data': '[[1517097600000,1]]', 'subreddit': 'AirSwap'},
 {'data': '[[1498780800000,3.0],[1498867200000,1.0],[1498953600000,4.0],[1499040000000,17.0],[1499126400000,21.0],[1499212800000,3.0],[1499299200000,3.0],[1499385600000,13.0],[1499472000000,6.0],[1499558400000,1.0],[1499644800000,0.0],[1499731200000,4.0],[1499817600000,5.0],[1499904000000,0.0],[1499990400000,4.0],[1500076800000,10.0],[1500163200000,4.0],[1500249600000,7.0],[1500336000000,2.0],[1500422400000,1.0],[1500508800000,9.0],[1500595200000,4.0],[1500681600000,5.0],[1500768000000,1.0],[1500854400000,2.0],[1500940800000,2.0],[1501027200000,9.0],[1501113600000,4.0],[1501200000000,3.0],[1501286400000,0.0],[1501372800000,0.0],[1501459200000,6.0],[1501545600000,11.0],[1501632000000,5.0],[1501718400000,4.0],[1501804800000,2.0],[1501891200000,1.0],[1501977600000,0.0],[1502064000000,0.0],[1502150400000,2.0],[1502236800000,8.0],[1502323200000,8.0],[1502409600000,4.0],[1502496000000,4.0],[1502582400000,5.0],[1502668800000,2.0],[1502

In [33]:
arr_data

[{'data': '[[1517097600000,1]]', 'subreddit': 'AirSwap'},
 {'data': '[[1498780800000,3.0],[1498867200000,1.0],[1498953600000,4.0],[1499040000000,17.0],[1499126400000,21.0],[1499212800000,3.0],[1499299200000,3.0],[1499385600000,13.0],[1499472000000,6.0],[1499558400000,1.0],[1499644800000,0.0],[1499731200000,4.0],[1499817600000,5.0],[1499904000000,0.0],[1499990400000,4.0],[1500076800000,10.0],[1500163200000,4.0],[1500249600000,7.0],[1500336000000,2.0],[1500422400000,1.0],[1500508800000,9.0],[1500595200000,4.0],[1500681600000,5.0],[1500768000000,1.0],[1500854400000,2.0],[1500940800000,2.0],[1501027200000,9.0],[1501113600000,4.0],[1501200000000,3.0],[1501286400000,0.0],[1501372800000,0.0],[1501459200000,6.0],[1501545600000,11.0],[1501632000000,5.0],[1501718400000,4.0],[1501804800000,2.0],[1501891200000,1.0],[1501977600000,0.0],[1502064000000,0.0],[1502150400000,2.0],[1502236800000,8.0],[1502323200000,8.0],[1502409600000,4.0],[1502496000000,4.0],[1502582400000,5.0],[1502668800000,2.0],[1502